In [ ]:
import fireopal
import qiskit
from qiskit_ibm_runtime import QiskitRuntimeService
import matplotlib.pyplot as plt
import os


In [3]:
shot_count = 2048


def draw_circuit(qasm_str: str):
    """Draws a QASM circuit."""
    circuit = qiskit.QuantumCircuit.from_qasm_str(qasm_str)
    display(circuit.draw(fold=-1))


def plot_bv_results(results, hidden_string, title=""):
    """Plot a probability histogram and highlight the hidden string."""
    bitstrings = sorted(results.keys())

    def to_probability(value, total):
        if isinstance(value, float):
            return value
        return value / total

    probabilities = [to_probability(results[b], shot_count) for b in bitstrings]
    plt.figure(figsize=(50, 5))
    bars = plt.bar(bitstrings, probabilities)
    plt.xticks(rotation=90)

    for index, bitstring in enumerate(bitstrings):
        if bitstring != hidden_string:
            bars[index].set_color("grey")

    plt.ylabel("Probability")
    plt.ylim([0, 1])
    plt.title(title)
    plt.show()

## Providing quantum circuit

In [ ]:
circuit_qasm = """OPENQASM 2.0;
include "qelib1.inc";
qreg q[12];
creg c[11];
x q[11];
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];
h q[7];
h q[8];
h q[9];
h q[10];
h q[11];
barrier q[0],q[1],q[2],q[3],q[4],q[5],q[6],q[7],q[8],q[9],q[10],q[11];
cx q[0],q[11];
cx q[1],q[11];
cx q[2],q[11];
cx q[3],q[11];
cx q[4],q[11];
cx q[5],q[11];
cx q[6],q[11];
cx q[7],q[11];
cx q[8],q[11];
cx q[9],q[11];
cx q[10],q[11];
barrier q[0],q[1],q[2],q[3],q[4],q[5],q[6],q[7],q[8],q[9],q[10],q[11];
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];
h q[7];
h q[8];
h q[9];
h q[10];
h q[11];
barrier q[0],q[1],q[2],q[3],q[4],q[5],q[6],q[7],q[8],q[9],q[10],q[11];
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
measure q[7] -> c[7];
measure q[8] -> c[8];
measure q[9] -> c[9];
measure q[10] -> c[10];
"""
draw_circuit(circuit_qasm)

In [ ]:
token = ""
instance = ""
credentials = fireopal.credentials.make_credentials_for_ibm_cloud(
    token=token, instance=instance
)

QiskitRuntimeService.save_account(
    channel="ibm_cloud", token=token, instance=instance, overwrite=True
)
service = QiskitRuntimeService()

In [ ]:
supported_devices = fireopal.show_supported_devices(credentials=credentials)[
    "supported_devices"
]
for name in supported_devices:
    print(name)

In [ ]:
backend_name = "ibm_perth"
print(f"Will run on backend: {backend_name}")

In [ ]:
validate_results = fireopal.validate(
    circuits=[circuit_qasm], credentials=credentials, backend_name=backend_name
)

if validate_results["results"] == []:
    print("No errors found.")
else:
    print("The following errors were found:")
    for error in validate_results["results"]:
        print(error)

In [ ]:
print(
    "Submitted the circuit to IBM. Note: there may be a delay in getting results due to IBM "
    "device queues. Check the status through instructions at "
    "https://cloud.ibm.com/docs/quantum-computing?topic=quantum-computing-results."
)
real_hardware_results = fireopal.execute(
    circuits=[circuit_qasm],
    shot_count=shot_count,
    credentials=credentials,
    backend_name=backend_name,
)

bitstring_results = real_hardware_results["results"]

In [ ]:
print(f"Success probability: {100 * bitstring_results[0]['11111111111']:.2f}%")
plot_bv_results(
    bitstring_results[0], hidden_string="11111111111", title=f"Fire Opal ($n=11$)"
)

## Comparing Fire Opal with Qiskit

In [ ]:
from qiskit_ibm_runtime import Sampler, Options

backend = service.backend(backend_name)
options = Options()
options.execution.shots = shot_count
sampler = Sampler(backend=backend, options=options)

circuit_qiskit = qiskit.QuantumCircuit.from_qasm_str(circuit_qasm)
ibm_result = sampler.run(circuit_qiskit).result()
ibm_probabilities = (
    ibm_result.quasi_dists[0]
    .nearest_probability_distribution()
    .binary_probabilities(num_bits=11)
)

print(f"Success probability: {100 * ibm_probabilities['11111111111']:.2f}%")
plot_bv_results(
    ibm_probabilities, hidden_string="11111111111", title=f"{backend_name} ($n=11$)"
)